## Model Evaluation 

In [25]:
# import dependencies 
import math 

In [26]:
# Define input variables  
groups = [[1, 2, 3], [1], [0, 2]] 
votes = [5, 5, 5, 5]

In [27]:
def group_memberships(groups, votes):
    """
    Define group memberships for each participant.
    :param: groups (list of lists): a list denotes the group and contains its members. 
    :param: votes (list): number of participant's votes for a given project proposal.
    :returns (list of lists): retunrs a list of group membersips for each participant.  
    """
    # let each entry in votes define a unique identifyer for an agent 
    group_memberships = [[] for _ in range(len(votes))]

    # add group memberships of agent i as a list to each entry identifying an agent 
    for i in range(len(groups)):
        for j in groups[i]:
            group_memberships[j].append(i)
        
    return group_memberships

In [28]:
group_memberships = group_memberships(groups, votes)
print(group_memberships)

[[2], [0, 1], [0, 2], [0]]


In [29]:
def common_group(agent_i, agent_j):
    """
    Define an identifyer indicating whether two participants share any common group. 
    :param: agent_i denotes a participant not equal to a participant called agent_j. 
    :param: agent_j denotes a participant not equal to a participant called agent_i.
    :returns (bool): returns true if two participants share a common group and false otherwise.  
    """
    common_group = any(group in group_memberships[agent_j] for group in group_memberships[agent_i])

    return common_group

In [30]:
def K(agent_i, group):
    """
    Define the weighting function that attenuates the votes of agent i given different group memberships.  
    :param: agent_i denotes a participant not equal to a participant called agent_j.
    :param: (integer): group denotes the group number.
    :returns: attenuated number of votes for a given project. 
    """
    if agent_i in group or any(common_group(agent_i, j) for j in group):
        return math.sqrt(votes[agent_i])
    else:
        return votes[agent_i]


In [37]:
def connection_oriented_cluster_match(groups, votes, group_memberships):
    """
    This function calculates the connection-oriented cluster match of votes 
    for each agent and returns the attenuated sum of votes for a given project proposal.
    param: groups (list of lists): a list denotes the group and contains its members.
    param: votes (list): number of participant's votes for a given project proposal.
    param: group_membership: defines group memberships for each participant.
    """ 
    weighted_votes = 0

    for group in groups:
        for agent_i in group:
            weighted_votes += votes[agent_i] / len(group_memberships[agent_i])

    for group in groups:
        for other_group in groups:
            if group == other_group:
                continue

            term1 = sum(K(agent_i, other_group) / len(group_memberships[agent_i]) for agent_i in group)
            term1 = math.sqrt(term1)

            term2 = sum(K(agent_j, group) / len(group_memberships[agent_j]) for agent_j in other_group)
            term2 = math.sqrt(term2)

            weighted_votes += term1 * term2

    return math.sqrt(weighted_votes)

In [38]:
result = connection_oriented_cluster_match(groups, votes, group_memberships)
print(result)

6.119546254329342


In [19]:
def cluster_match(groups, contributions):
    """
    groups: bag of groups (list of lists of integers)
    contributions: list of contributions (floats)
    group_memberships: list of sets (lists of lists) where
                        group_memberships[i] is the groups agent i belongs to
    """
    group_memberships = [[] for _ in range(len(contributions))]

    for i in range(len(groups)):
        for j in groups[i]:
            group_memberships[j].append(i)

    def common_group(i, j):
        """
        If voter i and voter j share any common group, return true. Else return false.
        """
        return any(group in group_memberships[j] for group in group_memberships[i])

    def K(i, group):
        """
        If group includes voter i, or any member of group shares a different group with voter i
        """
        if i in group or any(common_group(i, j) for j in group):
            return math.sqrt(contributions[i])
        return contributions[i]

    result = 0

    for g in groups:
        for i in g:
            result += contributions[i] / len(group_memberships[i])

    for g in groups:
        for h in groups:
            if g == h:
                continue

            term1 = sum(K(i, h) / len(group_memberships[i]) for i in g)
            term1 = math.sqrt(term1)

            term2 = sum(K(j, g) / len(group_memberships[j]) for j in h)
            term2 = math.sqrt(term2)

            result += term1 * term2

    return math.sqrt(result)

In [20]:
result = cluster_match(groups, contributions) 
print(result)

6.119546254329342
